# Xarray hands-on exercises

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import xarray as xr
import hvplot.xarray
import rioxarray
import zarr
from ipyleaflet import Map, basemaps, DrawControl, LayersControl
from ipyfastscape import TopoViz3d

from utils import extract_dem, run_fastscape_model

## Let's get some topographic data

Using an interactive map ([ipyleaflet](https://ipyleaflet.readthedocs.io)) to delimit a ROI, using the [bmi-topography](https://github.com/csdms/bmi-topography) package to download and extract elevation data (SRTM), and using [rioxarray](https://corteva.github.io/rioxarray/) to re-project the data from lat/lon to UTM.

In [ ]:
m = Map(basemap=basemaps.OpenTopoMap, center=[40, -105], zoom=11)

draw_control = DrawControl()
draw_control.polygon = {}
#draw_control.polyline = {}
draw_control.circlemarker = {}
draw_control.rectangle = {
    'shapeOptions': {
        'fillOpacity': 0.5
    }
}
m.add_control(draw_control)

layers_control = LayersControl(position='topright')
m.add_control(layers_control)
    
m

In [ ]:
def load_dem_from_map():
    """This will load DEM data using the last rectangle drawn on the map as ROI.
    
    Returns a new :class:`xarray.Dataset`.
    
    """
    last_draw = draw_control.last_draw['geometry']
    if last_draw is not None:
        roi = last_draw['coordinates'][0]
        lon, lat = list(zip(*roi))
        dem = extract_dem(north=max(lat), south=min(lat), east=max(lon), west=min(lon)).squeeze()
    else:
        dem = extract_dem().squeeze()
        
    projected = dem.rio.reproject(dem.rio.estimate_utm_crs()).isel(x=slice(2, -2), y=slice(2, -2))
    projected.attrs.pop("_FillValue")
    
    return projected

In [ ]:
dem = load_dem_from_map()

In [ ]:
dem

In [ ]:
dem.plot();

## Exercise 1: cross-sections

Extract and plot one or several cross-sections along the `x` or `y` axis.

Hints:

- use Xarray's `sel()` or `isel()`
- see Xarray's [plotting guide](http://xarray.pydata.org/en/stable/user-guide/plotting.html)

Create an interactive figure using `hvplot` where the position of the cross-section can be controlled with a slider

Extract a topographic profile given along a custom polyline defined from (`x`, `y`) points. Bonus: Draw a polyline on the map above and get it's lat/lon coordinates.

Hints:

- Use Xarray's [advanced indexing](http://xarray.pydata.org/en/stable/user-guide/indexing.html#more-advanced-indexing) (pointwise selection)

In [ ]:
lon = np.linspace(dem.x.min() + 300, dem.x.max() - 300, 50) + np.random.uniform(-300, 300, 50) 
lat = np.linspace(dem.y.min() + 300, dem.y.max() - 300, 50) + np.random.uniform(-300, 300, 50)

## Exercise 2: swath profiles

Extract and plot the mean/median/min/max elevation along the `x` or `y` axis. Bonus: gather all statistics into a single `xarray.DataArray` object and plot all the profiles with a legend using Xarray plotting methods.

Hints:

- use Xarray's [aggregation methods](http://xarray.pydata.org/en/stable/user-guide/computation.html#aggregation)
- See Xarray's [concatenate](http://xarray.pydata.org/en/stable/user-guide/combining.html#concatenate)

## Exercise 3: Compute terrain derivatives

Compute and plot terrain slope using the following formula:

$$ s = \arctan \left( \sqrt{\frac{\partial{z}}{\partial{x}}^2 + \frac{\partial{z}}{\partial{y}}^2} \right) $$

For simplicity, let's ignore the diagnonal DEM grid neighbors in the computation of the partial derivatives. Convert the values in degrees.

Hints:

- Look at Xarray's [differentiate](http://xarray.pydata.org/en/stable/generated/xarray.DataArray.differentiate.html#xarray.DataArray.differentiate)

## Let's run a landscape evolution model

In the example below we use [fastscape](https://github.com/fastscape-lem/fastscape) but the same exercises could be done from Landlab simulations as it also has an Xarray interface.  

In [ ]:
from dask.distributed import Client, LocalCluster
cluster = LocalCluster(threads_per_worker=1)
client = Client(cluster)

In [ ]:
client

In [ ]:
model_out = run_fastscape_model(dem, client, store="out.zarr")

In [ ]:
zdataset = zarr.open("out.zarr")

In [ ]:
zdataset.info

In [ ]:
zdataset.topography__elevation.info

In [ ]:
model_out

## Exercise 4

Re-do exercises 1-3 but using the model output dataset (`topography__elevation` variable).

## Exercise 5: Slope-Area plot

Plot slope vs. area relationship for one simulation and one time step. Bonus 1: make an interactive plot with `hvplot` for all simulations and all timesteps. Bonus 2: do not plot model grid nodes with a drainage area value under a given threshold. 